# Задание к теме «Гибридные рекомендательные системы»
Что делать?

1. Датасет ml-latest
2. Вспомнить подходы, которые мы разбирали
3. Выбрать понравившийся подход к гибридным системам
4. Написать свою

In [1]:
# Загружаем необходимые библиотеки
from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

In [2]:
# Загружаем данные
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
# Объединяем данные
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [4]:
# Составляем датасет для модели
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [5]:
# Определяем минимальный рейтинг
ratings.rating.min()

0.5

In [6]:
# Определяем максимальный рейтинг
ratings.rating.max()

5.0

In [7]:
# Определяем ридер и задаём масштаб значений
reader = Reader(rating_scale=(0.5, 5.0))

In [8]:
# Формируем датасет
data = Dataset.load_from_df(dataset, reader)

In [9]:
# Разделяем данные не тренировочную и тестовую части
trainset, testset = train_test_split(data, test_size=.15)

In [10]:
# Получаем скрытые факторы
algo = SVDpp(n_factors=20, n_epochs=20)
algo.fit(trainset)

In [11]:
# Осуществляем предсказания на тестовом датасете
test_pred = algo.test(testset)

In [12]:
# Определяем RMSE
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8593


0.8593213273761116

In [13]:
# Проскорим фильмы для дальнейшей работы
current_user_id = 7.0
user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in movies_with_ratings.title.unique():
    if movie in user_movies:
        continue
        
    scores.append(algo.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

In [14]:
# Смотрим рейтинг топ-10 фильмов
sorted(scores)[-10:]

[4.050582373299139,
 4.054810106462027,
 4.064213646540347,
 4.067417095655176,
 4.083271313174095,
 4.086665305460706,
 4.099690127609748,
 4.136483203632401,
 4.14548320101429,
 4.285408061492748]

In [15]:
# Функция, разделяющая строку на части
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [16]:
# Получаем теги
movie_genres = [change_string(g) for g in movies.genres.values]
movie_genres[0]

'Adventure Animation Children Comedy Fantasy'

In [17]:
# Подсчитываем количество слов
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

In [18]:
# Выполняем TF-IDF преобразование
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [19]:
# Находим ближайших соседей по полученным TF-IDF векторам
neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=20)

In [20]:
# Делаем предсказание
test = change_string("Adventure|Comedy|Fantasy|Crime")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [21]:
# Создаём словарь из наименования фильма (ключ) и тегов (значение)
title_genres = {}

for index, row in tqdm_notebook(movies.iterrows()):
    title_genres[row.title] = row.genres

C:\Users\khark\AppData\Local\Temp/ipykernel_15892/1046397591.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm_notebook(movies.iterrows()):


0it [00:00, ?it/s]

In [22]:
# Функция рекомендаций для пользователей на основе SVDpp и KDTree
def recommend_for_user(user_id):
    current_user_id = user_id # Фиксируем пользователя
    user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique() # Определяем фильмы, которые он видел
    
    last_user_movie = user_movies[-1] # Последний просмотренный фильм
    
    movie_genres = title_genres[last_user_movie] # Жанры последнего фильма
    
    movie_genres = change_string(movie_genres) # Разделяем жанры последнего фильма

    predict = count_vect.transform([movie_genres]) # Трансформируем в количество слов
    X_tfidf2 = tfidf_transformer.transform(predict) # Трансформируем в TF-IDF веса

    res = neigh.kneighbors(X_tfidf2, return_distance=True) # Получаем 20 близких по жанру фильмов
    
    movies_to_score = movies.iloc[res[1][0]].title.values # Сохраняем их в фильмы для скоринга

    scores = []
    titles = []

    # Определяем скор для каждого фильма, сохранённого для скора
    for movie in movies_to_score:
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=current_user_id, iid=movie).est)
        titles.append(movie)
        
    
    best_indexes = np.argsort(scores)[-10:] # Получаем лучшие 10 фильмов
    # Выводим топ 10 фильмов
    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [28]:
# Строим рекомендацию для пользователя
recommend_for_user(12.0)

Big Fish (2003) 4.8998158200203665
Like Water for Chocolate (Como agua para chocolate) (1992) 4.850195305524358
Wristcutters: A Love Story (2006) 4.7656883812688235
About Time (2013) 4.750849780377645
Ghost and Mrs. Muir, The (1947) 4.643931966106656
Double Life of Veronique, The (Double Vie de Véronique, La) (1991) 4.52236613705367
Orlando (1992) 4.517485125549778
Ghost (1990) 4.512233159730549
Wings of Desire (Himmel über Berlin, Der) (1987) 4.489072158746132
Edward Scissorhands (1990) 4.48749556844051


In [29]:
# Для проверки корректной работы функции посмотрим лучшие оценки от этого пользователя
movies_with_ratings[movies_with_ratings.userId == 12.0].sort_values('rating', ascending=False).head(10)

,movieId,title,genres,userId,rating,timestamp
39121,2072,"'burbs, The (1989)",Comedy,12.0,5.0,1.247264e+09
30336,1357,Shine (1996),Drama|Romance,12.0,5.0,1.247263e+09
81283,40629,Pride & Prejudice (2005),Drama|Romance,12.0,5.0,1.247264e+09
76352,8533,"Notebook, The (2004)",Drama|Romance,12.0,5.0,1.247264e+09
72984,6942,Love Actually (2003),Comedy|Drama|Romance,12.0,5.0,1.247264e+09
56154,3668,Romeo and Juliet (1968),Drama|Romance,12.0,5.0,1.247264e+09
47245,2717,Ghostbusters II (1989),Comedy|Fantasy|Sci-Fi,12.0,5.0,1.247263e+09
45404,2581,Never Been Kissed (1999),Comedy|Romance,12.0,5.0,1.247263e+09
45293,2572,10 Things I Hate About You (1999),Comedy|Romance,12.0,5.0,1.247264e+09
44223,2485,She's All That (1999),Comedy|Romance,12.0,5.0,1.247264e+09


Рекомендованные фильмы близки к тем, что пользователь высоко оценил. Следовательно, функция работает корректно.